# Geoapify Places 5 Cities

In [13]:
# Dependencies
import requests
import json

# Import the API key
from config_key import geoapify_key

In [14]:
# Set the geographical coordinates for Bronx, NY
cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco Bay Area, CA","Monmouth, OR"]

latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]

# Set the parameters for the type of place
# categories = ["catering.restaurant","commercial.supermarket"]
# conditions = "vegetarian"
categories = "catering.restaurant"
radius = 8000

# Set the parameters for the type of search
filters = f"circle:{longitude[2]},{latitude[2]},{radius}"
bias = f"proximity:{longitude[2]},{latitude[2]}"
limit = 100

# set up a parameters dictionary
params = {
    "categories":categories,
    #"conditions":conditions,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [15]:
# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)

https://api.geoapify.com/v2/places?categories=catering.restaurant&limit=100&filter=circle%3A-117.292763%2C34.536217%2C8000&bias=proximity%3A-117.292763%2C34.536217&apiKey=bf8e20276a9741e3b5a704ce3a508735


In [16]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -117.29501489999998,
                    34.53412204962398
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "El Super Pollo",
                "address_line2": "15521 7th Street, Victorville, CA 92395, United States of America",
                "categories": [
                    "building",
                    "building.catering",
                    "building.commercial",
                    "catering",
                    "catering.restaurant",
                    "catering.restaurant.chicken"
                ],
                "city": "Victorville",
                "country": "United States",
                "country_code": "us",
                "county": "San Bernardino County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                 

In [17]:
# Print the name and address of the first restaurant that appears
print(places_data["features"][0]["properties"]["name"])
print(places_data["features"][0]["properties"]["address_line2"])

El Super Pollo
15521 7th Street, Victorville, CA 92395, United States of America


In [18]:
from pprint import pprint
pprint(places_data)

{'features': [{'geometry': {'coordinates': [-117.29501489999998,
                                            34.53412204962398],
                            'type': 'Point'},
               'properties': {'address_line1': 'El Super Pollo',
                              'address_line2': '15521 7th Street, Victorville, '
                                               'CA 92395, United States of '
                                               'America',
                              'categories': ['building',
                                             'building.catering',
                                             'building.commercial',
                                             'catering',
                                             'catering.restaurant',
                                             'catering.restaurant.chicken'],
                              'city': 'Victorville',
                              'country': 'United States',
                              'country_c